In [7]:
import numpy as np
import math

In [76]:
def fit(X, y):
    N, D = X.shape
    cs = np.unique(y)
    C = len(cs)
    sigma2 = np.zeros(D)
    mus = []
    prior = []

    for c in cs:
        i = y == c
        X_c = X[i]
        prior.append(np.mean(i))
        mu_c = np.mean(X_c, axis = 0)
        mus.append(mu_c)
        for x in X_c:
            sigma2 += (x - mu_c) ** 2 / (1.0 * N - C)

    log_prior = np.log(prior)    
    return cs, log_prior, mus, sigma2
        
def predict(model, X):
    N, D = X.shape
    cs, log_prior, mus, sigma2 = model
    p = []
    
    for i in range(len(cs)):
        Z = 0.5 * (X - mus[i]) ** 2 / sigma2
        p.append(log_prior[i] - np.sum([Z[:, j] for j in range(D)], axis = 0))
    return np.array([cs[i] for i in np.argmax(np.transpose(p), axis = 1)])

In [77]:
X = np.array([
    [2, 3, 4, 5],
    [2.2, 3.1, 4.2, 5.3],
    [2.1, 3.2, 4.1, 5.2],
])

y = np.array([1, 1, 2])

In [78]:
model = fit(X, y)
np.mean(predict(model, X) == y)

1.0